In [1]:
#from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import torch

E:\Users\anaconda3\envs\py36\lib\site-packages\setuptools\distutils_patch.py:26: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  "Distutils was imported before Setuptools. This usage is discouraged "


## Preprocess dataset

In [27]:
import numpy as np # linear algebra
import pandas as pd 
names = ['male', 'age', 'debt', 'married', 'bankcustomer', 'educationlevel', 'ethnicity', 'yearsemployed',
               'priordefault', 'employed', 'creditscore', 'driverslicense', 'citizen', 'zip', 'income', 'approved']

data = pd.read_csv('./data/credit.csv', header=None,  names=names)
data.reset_index(drop=True, inplace=True) 


data = data.dropna(how = 'all')
data = data[data.age != '?']



from sklearn import preprocessing
for feat in ['male', 'married','bankcustomer', 'educationlevel', 'ethnicity','priordefault', 'employed', 'driverslicense', 'citizen', 'zip', 'approved']:
    data[feat] = preprocessing.LabelEncoder().fit_transform(data[feat])
    
data.head()

,male,age,debt,married,bankcustomer,educationlevel,ethnicity,yearsemployed,priordefault,employed,creditscore,driverslicense,citizen,zip,income,approved
0,2,30.83,0.000,2,1,13,8,1.25,1,1,1,0,0,68,0,0
1,1,58.67,4.460,2,1,11,4,3.04,1,1,6,0,0,11,560,0
2,1,24.50,0.500,2,1,11,4,1.50,1,0,0,0,0,96,824,0
3,2,27.83,1.540,2,1,13,8,3.75,1,1,5,1,0,31,3,0
4,2,20.17,5.625,2,1,13,8,1.71,1,0,0,0,2,37,0,0


## The dag used from paper

In [28]:
# Define DAG for Credit dataset
dag= [    
    # Edges from age
    ['age', 'yearsemployed'],
    
    # Edges from ethnicity
    ['ethnicity', 'approved'],
    ['ethnicity', 'married'],
    
    # Edges from default
    ["priordefault", "creditscore"],
    ["priordefault", "approved"],
    ["priordefault", "employed"],
    
    # Edges from zip
    ["zip", "married"],
    # Edges from citizen
    ["citizen","married"],
    # Edges from driverslicense
    ["driverslicense","employed"],
    # Edges from education_level
    ["educationlevel","employed"],
    ["educationlevel","married"],
    
    # Edges from yearsemployed
    ["yearsemployed", "creditscore"],
    # Edges from creditscore
    ["creditscore", "approved"],
    ["creditscore", "debt"],
    
    # Edges from employed
    ["employed", "bankcustomer"],
    ["employed", "debt"],
    
    # Edges from debt
    ["debt", "income"],
    # Edges from married
    ["married", "approved"],
    
    # Edges from income
    ["income", "approved"],
    ["income", "married"],
]

print(len(dag))
def dag_to_idx(df, dag):
    """Convert columns in a DAG to the corresponding indices."""

    dag_idx = []
    for edge in dag:
        dag_idx.append([df.columns.get_loc(edge[0]), df.columns.get_loc(edge[1])])

    return dag_idx

# Convert the DAG to one that can be provided to the DECAF model
dag_seed = dag_to_idx(data, dag)
print(dag_seed)

20
[[1, 7], [6, 15], [6, 3], [8, 10], [8, 15], [8, 9], [13, 3], [12, 3], [11, 9], [5, 9], [5, 3], [7, 10], [10, 15], [10, 2], [9, 4], [9, 2], [2, 14], [3, 15], [14, 15], [14, 3]]


## Use dag from notebook

In [30]:
from pycausal.pycausal import pycausal as pc
pc = pc()
pc.start_vm()

from pycausal import prior as p
prior = p.knowledge(addtemporal = [['male', 'age','ethnicity'],[ 'debt', 'married', 'bankcustomer', 'educationlevel', 'yearsemployed',
                'employed', 'creditscore', 'driverslicense', 'citizen', 'zip', 'income'],['approved']])


from pycausal import search as s
tetrad = s.tetradrunner()
tetrad.run(algoId = 'fges', scoreId = 'cg-bic-score', dfs = data, priorKnowledge = prior,
           maxDegree = -1, faithfulnessAssumed = True, verbose = False)
tetrad.getEdges()

dag_seed = []
for edge in tetrad.getEdges():
    dag_seed.append(list([names.index(edge.split(' ')[0]), names.index(edge.split(' ')[-1])]))
print(dag_seed )

[[1, 7], [8, 1], [10, 2], [6, 1], [7, 10], [13, 2], [3, 15], [7, 2], [9, 10], [4, 15], [9, 15], [8, 7], [4, 9], [4, 3], [8, 15], [7, 11], [12, 9], [10, 13], [9, 8]]


In [48]:
import pickle 
p_idx = 6
p_attr = 'ethnicity'
#for p in [0, 0.2, 0.4, 0.6, 0.8, 1]:
for p in [1]:
    names = ['male', 'age', 'debt', 'married', 'bankcustomer', 'educationlevel', 'ethnicity', 'yearsemployed',
               'priordefault', 'employed', 'creditscore', 'driverslicense', 'citizen', 'zip', 'income', 'approved']
    data = pd.read_csv('./data/credit.csv', header=None,  names=names)
    data.reset_index(drop=True, inplace=True) 
    data = data.dropna(how = 'all')

    data = data[data.age != '?']
    data.reset_index(drop=True, inplace = True)


    from sklearn import preprocessing
    for feat in ['male', 'married','bankcustomer', 'educationlevel', 'ethnicity','priordefault', 'employed', 'driverslicense', 'citizen', 'zip', 'approved']:
        data[feat] = preprocessing.LabelEncoder().fit_transform(data[feat])

    data['age'] = pd.to_numeric(data['age'],errors='coerce')


    data.loc[data['ethnicity'] <= 4, 'ethnicity'] = 0
    data.loc[data['ethnicity'] > 4, 'ethnicity']= 1
    
    
    data.loc[data['ethnicity'] ==1 , 'employed'] =  1

    biased_data = data.copy()
    
    
    bias = p
    biased_data.loc[biased_data['ethnicity'] == 1, 'approved'] = np.logical_and(biased_data.loc[biased_data['ethnicity'] == 1, 'approved'].values, 
                   np.random.binomial(1, bias, len(biased_data.loc[biased_data['ethnicity'] == 1, 'approved']))).astype(int)
    print(biased_data['approved'].value_counts())
    biased_data.head()

    thresh = 0.8

    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    scaler.fit(data)
    data[data.columns] = scaler.fit_transform(data)
    biased_data[biased_data.columns] = scaler.transform(biased_data)
    print(biased_data.head)
#     import pickle 
#     p_idx = 6
#     p_attr = 'ethnicity'

#     view_stats_new(['DECAF-FTU1'], biased_data, protected = p_attr, remove_protected = False,
#                orig_data = data ,protected_idx = p_idx, bias_dict ={15:[6]})
    
#     view_stats_new(['DECAF-FTU2'], biased_data, protected = p_attr, remove_protected = False,
#                orig_data = data ,protected_idx = p_idx, bias_dict ={15:[6]}, surrogate = True,)

#     view_stats_new(['DECAF-DP'], biased_data, protected = p_attr, remove_protected = False,
#                orig_data = data ,protected_idx = p_idx, bias_dict ={15:[6,9]})

1    373
0    305
Name: approved, dtype: int64
<bound method NDFrame.head of      male       age      debt   married  bankcustomer  educationlevel  \
0     1.0  0.256842  0.000000  0.666667      0.333333        0.928571   
1     0.5  0.675489  0.159286  0.666667      0.333333        0.785714   
2     0.5  0.161654  0.017857  0.666667      0.333333        0.785714   
3     1.0  0.211729  0.055000  0.666667      0.333333        0.928571   
4     1.0  0.096541  0.200893  0.666667      0.333333        0.928571   
..    ...       ...       ...       ...           ...             ...   
673   1.0  0.110226  0.360179  1.000000      1.000000        0.357143   
674   0.5  0.134135  0.026786  0.666667      0.333333        0.142857   
675   0.5  0.172932  0.482143  1.000000      1.000000        0.428571   
676   1.0  0.062707  0.007321  0.666667      0.333333        0.071429   
677   1.0  0.319549  0.120536  0.666667      0.333333        0.142857   

     ethnicity  yearsemployed  priordefault  e

In [49]:
dataset_train, dataset_test = train_test_split(biased_data, test_size=0.2,
                                               stratify=biased_data['approved'])
print(dataset_train.shape)

(542, 16)


In [50]:
from models.DECAF import DECAF
from data import DataModule
import os
models_dir = './cache/'
def train_decaf(train_dataset, dag_seed, biased_edges={}, h_dim=200, lr=0.5e-3,
                batch_size=64, lambda_privacy=0, lambda_gp=10, d_updates=10,
                alpha=2, rho=2, weight_decay=1e-2, grad_dag_loss=False, l1_g=0,
                l1_W=1e-4, p_gen=-1, use_mask=True, epochs=50):
    model_filename = os.path.join(models_dir, 'decaf.pkl')

    dm = DataModule(train_dataset.values)

    model = DECAF(
        dm.dims[0],
        dag_seed=dag_seed,
        h_dim=h_dim,
        lr=lr,
        batch_size=batch_size,
        lambda_privacy=lambda_privacy,
        lambda_gp=lambda_gp,
        d_updates=d_updates,
        alpha=alpha,
        rho=rho,
        weight_decay=weight_decay,
        grad_dag_loss=grad_dag_loss,
        l1_g=l1_g,
        l1_W=l1_W,
        p_gen=p_gen,
        use_mask=use_mask,
    )
    print(model_filename)
    if os.path.exists(model_filename):
        model = torch.load(model_filename)
    else:
        trainer = pl.Trainer(max_epochs=epochs, logger=False)
        trainer.fit(model, dm)
        #torch.save(model, model_filename)

    # Generate synthetic data
    synth_dataset = (
        model.gen_synthetic(
            dm.dataset.x,
            gen_order=model.get_gen_order(),
            biased_edges=biased_edges,
        )
        .detach()
        .numpy()
    )
    synth_dataset[:, -1] = synth_dataset[:, -1].astype(np.int8)

    synth_dataset = pd.DataFrame(synth_dataset,
                                 index=train_dataset.index,
                                 columns=train_dataset.columns)
    #synth_dataset['approved'] = np.round(synth_dataset['approved'])
    #synth_dataset['income'] = np.round(synth_dataset['income'])

    return synth_dataset

In [51]:
synth_data = train_decaf(biased_data, dag_seed)

E:\Users\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\core\datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
E:\Users\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\core\datamodule.py:170: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
E:\Users\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. 

Initialised adjacency matrix as parsed:
 Parameter containing:
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 

E:\Users\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:631: UserWarning: Checkpoint directory D:\Github\UvA_FACT2022_old\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
E:\Users\anaconda3\envs\py36\lib\site-packages\pytorch_lightning\trainer\data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


In [52]:
synth_data.head()

,male,age,debt,married,bankcustomer,educationlevel,ethnicity,yearsemployed,priordefault,employed,creditscore,driverslicense,citizen,zip,income,approved
0,0.869452,0.245637,0.143743,0.760586,0.530070,0.466076,0.672207,0.087154,0.489890,0.775127,0.049974,0.472785,0.151772,0.279125,0.099935,0.0
1,0.811601,0.188997,0.105698,0.759693,0.529114,0.465543,0.703712,0.107807,0.490897,0.794090,0.049619,0.468848,0.064284,0.292168,0.049855,0.0
2,0.884090,0.198660,0.149372,0.730109,0.529360,0.465994,0.677262,0.082699,0.490567,0.775656,0.034285,0.468821,0.122167,0.297976,0.048711,0.0
3,0.785310,0.183638,0.135238,0.756131,0.529453,0.466070,0.630915,0.069631,0.490491,0.800056,0.054056,0.462593,0.064196,0.292962,0.065589,0.0
4,0.865611,0.218471,0.110062,0.727579,0.527782,0.463618,0.664468,0.072263,0.490419,0.755757,0.049868,0.472785,0.065033,0.339460,0.060757,0.0


In [53]:
def eval_model(dataset_train, dataset_test, label="approved"):
    """Helper function that prints evaluation metrics."""

    X_train, y_train = dataset_train.drop(columns=[label]), dataset_train[label]
    X_test, y_test = dataset_test.drop(columns=[label]), dataset_test[label]

    clf = MLPClassifier()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    auroc = roc_auc_score(y_test, y_pred)
    dp = DP(clf, X_test)
    ftu = FTU(clf, X_test)

    return {'precision': precision, 'recall': recall, 'auroc': auroc,
            'dp': dp, 'ftu': ftu}

def DP(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1]) 
    X_test_0 = X_test_df[X_test_df["ethnicity"] <0.5]
    X_test_1 = X_test_df[X_test_df["ethnicity"] >0.5]
    dp = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))
    
    return dp
    

def FTU(mlp, X_test):
    """Calculate fairness metric DP"""
    
    X_test_df = pd.DataFrame(X_test, columns = names[:-1])
    X_test_0 = X_test_df.assign(ethnicity = 0)
    X_test_1 = X_test_df.assign(ethnicity = 1)

    ftu = abs(np.mean(mlp.predict(X_test_0)) - np.mean(mlp.predict(X_test_1)))

    return ftu

In [54]:

print(synth_data["approved"].describe())
#synth_data.hist()
#dataset_test

count    678.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: approved, dtype: float64


In [55]:
eval_model(synth_data, dataset_test)

E:\Users\anaconda3\envs\py36\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.0, 'recall': 0.0, 'auroc': 0.5, 'dp': 0.0, 'ftu': 0.0}